In [1]:
import pickle
import random
import json

import pandas as pd
import numpy as np 

import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import classification_report, f1_score, precision_score, recall_score


import xgboost as xgb, lightgbm as lgbm, catboost as catb
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score, ShuffleSplit, learning_curve
from sklearn import svm

In [2]:
!pip install flask_ngrok

In [3]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

In [4]:
TRAIN_PATH = 'C:/Users/Anatoly/Desktop/Python_and_DS/course_project/course_project_train/course_project_train.csv'
TEST_PATH = 'C:/Users/Anatoly/Desktop/Python_and_DS/course_project/course_project_test/course_project_test.csv'

In [5]:
df = pd.read_csv(TRAIN_PATH)

In [6]:
df.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [7]:
TARGET_NAME = 'Credit Default'

In [8]:
df.loc[23]

Home Ownership                            Own Home
Annual Income                                  NaN
Years in current job                     10+ years
Tax Liens                                        0
Number of Open Accounts                         17
Years of Credit History                       14.8
Maximum Open Credit                         700040
Number of Credit Problems                        0
Months since last delinquent                    17
Bankruptcies                                     0
Purpose                         debt consolidation
Term                                    Short Term
Current Loan Amount                         174878
Current Credit Balance                      168074
Monthly Debt                                 17132
Credit Score                                   NaN
Credit Default                                   1
Name: 23, dtype: object

In [9]:
INITIAL_FEATURES = ["Home Ownership","Annual Income", "Years in current job","Tax Liens",
      "Number of Open Accounts","Years of Credit History","Maximum Open Credit",
      "Number of Credit Problems","Months since last delinquent","Purpose","Term",
      "Current Loan Amount","Current Credit Balance","Monthly Debt","Credit Score"]

In [10]:
len(INITIAL_FEATURES)

15

In [11]:
print(INITIAL_FEATURES)

['Home Ownership', 'Annual Income', 'Years in current job', 'Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 'Number of Credit Problems', 'Months since last delinquent', 'Purpose', 'Term', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt', 'Credit Score']


Намеренно удалил все ненужное. Сфокусируемся на преобразовании фичей для модели. 

In [12]:
def feature_transformation(df):
    """Инжиниринг данных"""

    # Исправляем выбросы по Credit Score - очевидно что там лишний ноль дописали (+) 
    df['Credit Score'] = df['Credit Score'].fillna(0)
    df.loc[df['Credit Score'] < 300, 'Credit Score'] = 0
    df.loc[df['Credit Score'] > 850, 'Credit Score'] = df['Credit Score'] // 10
    
    # Monthly debt
    df.loc[df['Monthly Debt'] > 50000, 'Monthly Debt'] = df['Monthly Debt'].median()
    
    # Current Credit Balance
    df.loc[df['Current Credit Balance'] > 1250000, 'Current Credit Balance'] = df['Current Credit Balance'].median()
    
    # Term - бинарный признак (+)
    
    df.loc[df['Term'] == 'Long Term', 'Term'] = 1
    df.loc[df['Term'] == 'Short Term', 'Term'] = 0
    
    # для Purpose (+):
    
    for purpose in df['Purpose'].unique(): 
        if purpose == 'debt consolidation':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = -1
        elif purpose == 'other':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 0 
        elif purpose == 'home improvements':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 1 
        else:
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 2
    
    # Банкротства (+)
    #df['Bankruptcies'] = df['Bankruptcies'].fillna(df['Bankruptcies'].mode()[0])
    #df.loc[df['Bankruptcies'] > 1, 'Bankruptcies'] = 1

    # Вместо Nan в "Months since last delinquent" ставим 0 (+):
    
    df['Months since last delinquent'] = df['Months since last delinquent'].fillna(0)
    df.loc[df['Months since last delinquent'] > 100, 'Months since last delinquent'] = df['Months since last delinquent'].median()
    
    # Number of credit problems (+): 
    #df.loc[df['Number of Credit Problems'] > 1, 'Number of Credit Problems'] = 1
    
    # Maximum Open Credit (+):
    df.loc[df['Maximum Open Credit'] > 1300000, 'Maximum Open Credit'] = df['Maximum Open Credit'].median()
    
    #Years of Credit History
    df.loc[df['Years of Credit History'] > 47, 'Years of Credit History'] = df['Years of Credit History'].median()
    
    #Number of Open Accounts
    df.loc[df['Number of Open Accounts'] > 23, 'Number of Open Accounts'] = df['Number of Open Accounts'].median()
    
    # Tax liens (+): 
    df.loc[df['Tax Liens'] > 1, 'Tax Liens'] = 1

    # Правим Years in current job (+)
    df['Years in current job'] = df['Years in current job'].fillna(0)
    # В цикле отрабатываем все значения
    for year in df['Years in current job'].unique(): 
        if year in [0, '< 1 year']:
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = 0
        elif year == '10+ years':
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = 15 
        else:
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = int(year.split(' ')[0])
         
    # 'Home Ownership'
    for cat_feature in ['Home Ownership']:
        df = pd.concat([df, pd.get_dummies(df[cat_feature], prefix=cat_feature)], axis=1)
        
    return df

In [14]:
df = feature_transformation(df)

In [15]:
# Annual income

In [16]:
def input_variance(df, new_var, col, variance):
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_absolute_error
    from sklearn.preprocessing import StandardScaler

    rrf = RandomForestRegressor()
    sclx = StandardScaler()
    scly = StandardScaler()
    
    df[new_var] = df[variance].values
    
    if variance in col:
        col.remove(variance)
    
    X_train_test_predict = df[col].iloc[:,:]
    y_train_test_predict = df[new_var].iloc[:]
    
    index_nan = np.where(y_train_test_predict.isna()==True)[0]
    X_predict= X_train_test_predict.iloc[index_nan,:]
    
    index_test_train = np.where(y_train_test_predict.isna() == False)[0]
    X_train_test = X_train_test_predict.iloc[index_test_train,:]
    y_train_test = y_train_test_predict.iloc[index_test_train]
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size=0.2)
    
    sclx.fit(X_train)
    scly.fit(y_train.values.reshape((-1,1)))
    rrf.fit(sclx.transform(X_train), scly.transform(y_train.values.reshape((-1,1))))
    
    y_val = scly.inverse_transform(rrf.predict(sclx.transform(X_test)))
    print(mean_absolute_error(y_val,y_test))
    
    df[new_var].values[index_nan]=scly.inverse_transform(rrf.predict(sclx.transform(X_predict)))

    return df[new_var]

In [17]:
test_col = ['Tax Liens', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
            'Months since last delinquent', 'Term', 'Current Loan Amount', 'Current Credit Balance',
            'Monthly Debt', 'Credit Score', 'Classify_purpose', 'Number of years in current job', 
            'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Own Home', 
            'Home Ownership_Rent']

target_col = ['Credit Default']

train_col = test_col + target_col

In [18]:
df['new_income'] = input_variance(df = df, new_var='new_income', col = train_col, 
                                            variance = 'Annual Income')

419221.7791000841


In [19]:
FEATURES = ['new_income', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
            'Months since last delinquent', 'Term', 'Current Loan Amount', 'Current Credit Balance',
            'Monthly Debt', 'Credit Score', 'Classify_purpose', 'Number of years in current job', 
            'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Own Home', 
            'Home Ownership_Rent']

In [20]:
#['new_income', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
#            'Months since last delinquent', 'Term', 'Current Loan Amount', 'Current Credit Balance',
#            'Monthly Debt', 'Credit Score', 'Classify_purpose', 'Number of years in current job', 
#            'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Own Home', 
#            'Home Ownership_Rent']

In [21]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [22]:
from sklearn.model_selection import train_test_split

X = df[FEATURES]
y = df['Credit Default']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=True, test_size=0.3, random_state=42)

In [23]:
X_train.head()

,new_income,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Months since last delinquent,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Classify_purpose,Number of years in current job,Home Ownership_Have Mortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent
7179,3709940.00,5.0,15.4,899030.0,0.0,0,429572.0,277191.0,16076.5,748.0,-1.0,5.0,0,1,0,0
6054,1359418.84,21.0,27.5,1063040.0,70.0,0,357346.0,255227.0,7827.0,0.0,-1.0,15.0,0,1,0,0
7269,1874160.00,16.0,11.5,196856.0,75.0,0,130196.0,21413.0,37952.0,737.0,1.0,2.0,0,1,0,0
4866,2395026.00,10.0,13.7,316294.0,0.0,0,712250.0,243124.0,31135.0,717.0,-1.0,15.0,0,1,0,0
3593,631712.00,16.0,21.0,686928.0,0.0,1,212784.0,231382.0,9076.0,738.0,-1.0,4.0,0,0,1,0


In [24]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42, max_features=3, min_samples_leaf=2)
model_rf.fit(X_train, y_train)

y_train_pred = model_rf.predict(X_train)
y_test_pred = model_rf.predict(X_valid)

get_classification_report(y_train, y_train_pred, y_valid, y_test_pred)

TRAIN

              precision    recall  f1-score   support

           0       0.72      1.00      0.84      3795
           1       0.00      0.00      0.00      1455

    accuracy                           0.72      5250
   macro avg       0.36      0.50      0.42      5250
weighted avg       0.52      0.72      0.61      5250

TEST

              precision    recall  f1-score   support

           0       0.71      1.00      0.83      1592
           1       0.00      0.00      0.00       658

    accuracy                           0.71      2250
   macro avg       0.35      0.50      0.41      2250
weighted avg       0.50      0.71      0.59      2250

CONFUSION MATRIX

col_0              0
Credit Default      
0               1592
1                658


In [25]:
with open('model_rf.pkl', 'wb') as file:
    pickle.dump(model_rf, file)

In [26]:
loaded_model = pickle.load(open('model_rf.pkl', 'rb'))

In [41]:
SEL_FEATURES = ["Home Ownership",
                "Annual Income",
                "Years in current job",
                "Tax Liens",
                "Number of Open Accounts",
                "Years of Credit History",
                "Maximum Open Credit",
                "Number of Credit Problems", 
                "Months since last delinquent",
                "Purpose",
                "Term", 
                "Current Loan Amount",
                "Current Credit Balance",
                "Monthly Debt",
                "Credit Score"
               ]

In [112]:
json1 = {'ID':23, "Home Ownership":"Rent","Annual Income":164562.0,"Years in current job":"2 years","Tax Liens":0.0,
      "Number of Open Accounts":9.0,"Years of Credit History":12.5,"Maximum Open Credit":220968.0,
      "Number of Credit Problems":0.0,"Months since last delinquent":70.0,"Purpose":"debt consolidation","Term":"Short Term",
      "Current Loan Amount":162470.0,"Current Credit Balance":105906.0,"Monthly Debt":6813.0,"Credit Score":585.0}

json2 = {'ID':2,"Home Ownership":"Have Mortgage", "Annual Income":113186.0,"Years in current job":"10+ years","Tax Liens":0.0,
       "Number of Open Accounts":16.0,"Years of Credit History":17.0,"Maximum Open Credit":456302.0,
       "Number of Credit Problems":0.0,"Months since last delinquent":70.0,"Purpose":"debt consolidation",
       "Term":"Short Term","Current Loan Amount":217382.0,"Current Credit Balance":213199.0,"Monthly Debt":27559.0}

json3 = {'ID':3,"Home Ownership":"Home Mortgage","Annual Income":1152540.0,"Years in current job":"2 years","Tax Liens":0.0,
       "Number of Open Accounts":10.0,"Years of Credit History":13.7,"Maximum Open Credit":204600.0,
       "Number of Credit Problems":0.0,"Months since last delinquent":0.0,"Purpose":"debt consolidation","Term":"Short Term",
       "Current Loan Amount":200178.0,"Current Credit Balance":146490.0,"Monthly Debt":18729.0,"Credit Score":7260.0}

json4 = {'ID':4,"Home Ownership":"Own Home","Annual Income":114380.0,"Years in current job":"5 years","Tax Liens":0.0,
       "Number of Open Accounts":9.0,"Years of Credit History":9.7,"Maximum Open Credit":104600.0,
       "Number of Credit Problems":0.0,"Months since last delinquent":0.0,"Purpose":"debt consolidation","Term":"Short Term",
       "Current Loan Amount":100178.0,"Current Credit Balance":148490.0,"Monthly Debt":28729.0,"Credit Score":7260.0}



In [113]:
ddf = pd.DataFrame(columns = SEL_FEATURES)

In [122]:
DDF = ddf.append(pd.DataFrame([json1, json2, json3, json4],index=['23', '2', '3', '4'],columns=SEL_FEATURES))

In [115]:
DDF

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score
23,Rent,164562.0,2 years,0.0,9.0,12.5,220968.0,0.0,70.0,debt consolidation,Short Term,162470.0,105906.0,6813.0,585.0
2,Have Mortgage,113186.0,10+ years,0.0,16.0,17.0,456302.0,0.0,70.0,debt consolidation,Short Term,217382.0,213199.0,27559.0,NaN
3,Home Mortgage,1152540.0,2 years,0.0,10.0,13.7,204600.0,0.0,0.0,debt consolidation,Short Term,200178.0,146490.0,18729.0,7260.0
4,Own Home,114380.0,5 years,0.0,9.0,9.7,104600.0,0.0,0.0,debt consolidation,Short Term,100178.0,148490.0,28729.0,7260.0


In [116]:
ddf_trans = feature_transformation(DDF)

In [117]:
DDF_trans = ddf_trans.rename(columns={'Annual Income': 'new_income'})

In [118]:
DDF_trans

,Home Ownership,new_income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Purpose,...,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Classify_purpose,Number of years in current job,Home Ownership_Have Mortgage,Home Ownership_Home Mortgage,Home Ownership_Own Home,Home Ownership_Rent
23,Rent,164562.0,2 years,0.0,9.0,12.5,220968.0,0.0,70.0,debt consolidation,...,162470.0,105906.0,6813.0,585.0,-1.0,2.0,0,0,0,1
2,Have Mortgage,113186.0,10+ years,0.0,16.0,17.0,456302.0,0.0,70.0,debt consolidation,...,217382.0,213199.0,27559.0,0.0,-1.0,15.0,1,0,0,0
3,Home Mortgage,1152540.0,2 years,0.0,10.0,13.7,204600.0,0.0,0.0,debt consolidation,...,200178.0,146490.0,18729.0,726.0,-1.0,2.0,0,1,0,0
4,Own Home,114380.0,5 years,0.0,9.0,9.7,104600.0,0.0,0.0,debt consolidation,...,100178.0,148490.0,28729.0,726.0,-1.0,5.0,0,0,1,0


In [119]:
features = ['new_income', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit', 'Months since last delinquent', 'Term', 'Current Loan Amount', 'Current Credit Balance', 'Monthly Debt', 'Credit Score', 'Classify_purpose', 'Number of years in current job', 'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Own Home', 'Home Ownership_Rent']

In [120]:
y_pred = loaded_model.predict(DDF_trans[features])
y_pred_proba = loaded_model.predict_proba(DDF_trans[features])
y_pred, y_pred_proba

(array([0, 0, 0, 0], dtype=int64), array([[0.68931949, 0.31068051],
        [0.69845735, 0.30154265],
        [0.72084057, 0.27915943],
        [0.68916544, 0.31083456]]))

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

def transform(df):
    """Инжиниринг данных"""

    # Исправляем выбросы по Credit Score - очевидно что там лишний ноль дописали (+) 
    df['Credit Score'] = df['Credit Score'].fillna(0)
    df.loc[df['Credit Score'] < 300, 'Credit Score'] = 0
    df.loc[df['Credit Score'] > 850, 'Credit Score'] = df['Credit Score'] // 10
    
    # Monthly debt
    df.loc[df['Monthly Debt'] > 50000, 'Monthly Debt'] = df['Monthly Debt'].median()
    
    # Current Credit Balance
    df.loc[df['Current Credit Balance'] > 1250000, 'Current Credit Balance'] = df['Current Credit Balance'].median()
    
    # Term - бинарный признак (+)
    
    df.loc[df['Term'] == 'Long Term', 'Term'] = 1
    df.loc[df['Term'] == 'Short Term', 'Term'] = 0
    
    # для Purpose (+):
    
    for purpose in df['Purpose'].unique(): 
        if purpose == 'debt consolidation':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = -1
        elif purpose == 'other':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 0 
        elif purpose == 'home improvements':
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 1 
        else:
            df.loc[df['Purpose'] == purpose, 'Classify_purpose'] = 2
    
    # Банкротства (+)
    #df['Bankruptcies'] = df['Bankruptcies'].fillna(df['Bankruptcies'].mode()[0])
    #df.loc[df['Bankruptcies'] > 1, 'Bankruptcies'] = 1

    # Вместо Nan в "Months since last delinquent" ставим 0 (+):
    
    df['Months since last delinquent'] = df['Months since last delinquent'].fillna(0)
    df.loc[df['Months since last delinquent'] > 100, 'Months since last delinquent'] = df['Months since last delinquent'].median()
    
    # Number of credit problems (+): 
    #df.loc[df['Number of Credit Problems'] > 1, 'Number of Credit Problems'] = 1
    
    # Maximum Open Credit (+):
    df.loc[df['Maximum Open Credit'] > 1300000, 'Maximum Open Credit'] = df['Maximum Open Credit'].median()
    
    #Years of Credit History
    df.loc[df['Years of Credit History'] > 47, 'Years of Credit History'] = df['Years of Credit History'].median()
    
    #Number of Open Accounts
    df.loc[df['Number of Open Accounts'] > 23, 'Number of Open Accounts'] = df['Number of Open Accounts'].median()
    
    # Tax liens (+): 
    df.loc[df['Tax Liens'] > 1, 'Tax Liens'] = 1

    # Правим Years in current job (+)
    df['Years in current job'] = df['Years in current job'].fillna(0)
    # В цикле отрабатываем все значения
    for year in df['Years in current job'].unique(): 
        if year in [0, '< 1 year']:
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = 0
        elif year == '10+ years':
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = 15 
        else:
            df.loc[df['Years in current job'] == year, 'Number of years in current job'] = int(year.split(' ')[0])
         
    # 'Home Ownership'
    for cat_feature in ['Home Ownership']:
        df = pd.concat([df, pd.get_dummies(df[cat_feature], prefix=cat_feature)], axis=1)
        
    return df


SEL_FEATURES = ["Home Ownership",
                "Annual Income",
                "Years in current job",
                "Tax Liens",
                "Number of Open Accounts",
                "Years of Credit History",
                "Maximum Open Credit",
                "Number of Credit Problems", 
                "Months since last delinquent",
                "Purpose",
                "Term", 
                "Current Loan Amount",
                "Current Credit Balance",
                "Monthly Debt",
                "Credit Score"
               ]

@app.route('/predict', methods=['GET', 'POST'])
def predict3():
    try:

        json_input = request.json
#        print(json_input)
        ID = json_input["ID"]
        ddf = pd.DataFrame(columns = SEL_FEATURES)
        
        DDF = ddf.append(pd.DataFrame([json_input],index=[ID],columns=SEL_FEATURES))
        print(DDF)
        ddf_trans = feature_transformation(DDF)
        print(ddf_trans)
        
        DDF_trans = ddf_trans.rename(columns={'Annual Income': 'new_income'})
        
        features = ['new_income', 'Number of Open Accounts', 'Years of Credit History', 'Maximum Open Credit',
                    'Months since last delinquent', 'Term', 'Current Loan Amount', 'Current Credit Balance', 
                    'Monthly Debt', 'Credit Score', 'Classify_purpose', 'Number of years in current job',
                    'Home Ownership_Have Mortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Own Home',
                    'Home Ownership_Rent'
                   ]
        
        y_pred = loaded_model.predict(DDF_trans[features])
        print(y_pred)
        y_pred_proba = loaded_model.predict_proba(DDF_trans[features])
        print(y_pred, y_pred_proba)

#         return json.dumps({'ID': ID, 'Credit Default': int(y_pred[0]) , 'Credit Default_probability_1': float(y_pred_proba[0,1])}) 
        return jsonify({'ID': ID, 'Credit Default': int(y_pred[0]) , 'Credit Default_probability_1': float(y_pred_proba[0,1])}) 
    except:

        return "Error"



if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://34582b634da9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Dec/2020 21:58:42] "POST /predict HTTP/1.1" 200 -


   Home Ownership  Annual Income Years in current job  Tax Liens  \
23           Rent            0.0              2 years        0.0   

    Number of Open Accounts  Years of Credit History  Maximum Open Credit  \
23                      9.0                     12.5             220968.0   

    Number of Credit Problems  Months since last delinquent  \
23                        0.0                          70.0   

               Purpose        Term  Current Loan Amount  \
23  debt consolidation  Short Term             162470.0   

    Current Credit Balance  Monthly Debt  Credit Score  
23                105906.0        6813.0         585.0  
   Home Ownership  Annual Income Years in current job  Tax Liens  \
23           Rent            0.0              2 years        0.0   

    Number of Open Accounts  Years of Credit History  Maximum Open Credit  \
23                      9.0                     12.5             220968.0   

    Number of Credit Problems  Months since last delinqu